# 功能描述
- 遍历目录下所有网关LOG文件，锁定60 61传感器的各种上报
- 去除同一时间点的多次上报
- 相邻的点不应为相同值，如果相同应该为485通信丢包
- 仅关注6月23日以后

In [1]:
from datetime import datetime
import os
from os import path

In [2]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [3]:
BATHROOM = 0
BEDROOM = 1
ENTRY_DIFF_LIMIT = 1

In [4]:
time_start_point = datetime(2022, 6, 23, 0, 0, 0, 0)
bathroom_target_string = "485 induc,id:61,sta:1"
bathroom_notarget_string = "485 induc,id:61,sta:0"

bedroom_target_string = "485 induc,id:60,sta:1"
bedroom_notarget_string = "485 induc,id:60,sta:0"

room_last_target_tick = 0
room_last_notarget_tick = 0

In [5]:
def check_radar(file_name, room):
    global room_last_target_tick
    global room_last_notarget_tick
    room_list = []
    with open(file_name, 'r') as f:
        index = 0
        for line in f:
            index += 1
            if room == BATHROOM:
                temp_result1 = line.find(bathroom_target_string)
                temp_result2 = line.find(bathroom_notarget_string)
            elif room == BEDROOM:
                temp_result1 = line.find(bedroom_target_string)
                temp_result2 = line.find(bedroom_notarget_string)
            if temp_result1 >= 0:
                pos1 = line.find('[') + 1
                pos2 = line.find(']')
                date_time_str = line[pos1:pos2].strip()
                date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d-%H-%M-%S')
                was_after_data = time_start_point < date_time_obj
                if was_after_data:
                    last_diff = date_time_obj.timestamp() - room_last_target_tick
                    if last_diff > ENTRY_DIFF_LIMIT:
                        room_list.append((index, 0xff, date_time_obj))
                    room_last_target_tick = date_time_obj.timestamp()
            elif temp_result2 >= 0:
                pos1 = line.find('[') + 1
                pos2 = line.find(']')
                date_time_str = line[pos1:pos2].strip()
                date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d-%H-%M-%S')
                was_after_data = time_start_point < date_time_obj
                if was_after_data:
                    last_diff = date_time_obj.timestamp() - room_last_notarget_tick
                    if last_diff > ENTRY_DIFF_LIMIT:
                        room_list.append((index, 0x00, date_time_obj))
                    room_last_notarget_tick = date_time_obj.timestamp()
    #print(room_list)
    #print(len(room_list))
    
    fail_counter = 0
    if len(room_list) != 0:
        index_list, result_list, datatime_list = zip(*room_list)

        index_list = list(index_list)
        result_list = list(result_list)
        datatime_list = list(datatime_list)

        for x in range(0, len(result_list) - 1):
            if result_list[x] == result_list[x+1]:
                print(f'{bcolors.OKBLUE}{index_list[x]}@{datatime_list[x]}--{index_list[x+1]}@{datatime_list[x+1]}{bcolors.ENDC}')
                fail_counter = fail_counter + 1
        if fail_counter != 0:
            print(f'{bcolors.OKBLUE}'+'loss rate: {:.2f}%'.format(100*fail_counter/len(room_list))+f'{bcolors.ENDC}')
            print(file_name)
    else:
        fail_counter = 0
    return len(room_list), fail_counter

In [6]:
def scan_files(url, room):
    if room == BATHROOM:
        print(f'begin bathroom check')
    elif room == BEDROOM:
        print(f'begin bedroom check')
    file  = os.listdir(url)
    all_try_num = 0
    all_fail_num = 0
    for f in file:
        real_url = path.join(url, f)
        if path.isfile(real_url):
            #print(path.abspath(real_url))
            each_try_num, each_fail_num = check_radar(path.abspath(real_url), room)
            all_try_num += each_try_num
            all_fail_num += each_fail_num
            # 如果是文件，则以绝度路径的方式输出
        elif path.isdir(real_url):
            #如果是目录，则是递归自定义函数 scaner_file (url)进行多次
            scaner_file(real_url)
        else: 
            print("其他情况")
            pass
    print(f'{bcolors.OKBLUE}'+'room all loss rate: {:.2f}%'.format(100*all_fail_num/all_try_num)+f'{bcolors.ENDC}')

In [7]:
scan_files("./data", BATHROOM)

begin bathroom check
6353@2022-06-24 08:34:38--6354@2022-06-24 08:34:40
loss rate: 0.35%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\206.txt
7979@2022-06-24 09:48:59--7980@2022-06-24 09:49:01
loss rate: 0.36%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\208.txt
9335@2022-06-24 13:21:38--9336@2022-06-24 13:21:40
loss rate: 0.29%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\209.txt
4916@2022-06-23 17:08:18--4926@2022-06-23 17:43:31
loss rate: 3.70%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\233.txt
11112@2022-06-23 08:41:38--11113@2022-06-23 08:41:46
loss rate: 2.70%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\237.txt
3941@2022-06-23 14:08:43--3942@2022-06-23 14:08:45
loss rate: 0.90%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\306.txt
1948@2022-06-23 09:59:58--1949@2022-06-23 10:00:00
loss rate: 2.56%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\311.txt
3108@2022-06-23 13:49:52--3135@2022-06-23 13:51:24
3135@2022-06-23 13:51:2

In [8]:
scan_files("./data", BEDROOM)

begin bedroom check
8431@2022-06-23 10:50:21--8435@2022-06-23 10:50:23
loss rate: 2.63%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\209.txt
1566@2022-06-23 13:22:26--1570@2022-06-23 13:22:28
loss rate: 5.26%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\223.txt
6316@2022-06-23 10:43:20--6332@2022-06-23 10:45:24
loss rate: 3.03%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\227.txt
4908@2022-06-23 17:03:48--4927@2022-06-23 17:43:32
loss rate: 4.00%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\233.txt
4646@2022-06-23 12:07:25--4674@2022-06-23 12:09:45
loss rate: 3.70%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\236.txt
11181@2022-06-23 12:00:51--11223@2022-06-23 12:04:33
loss rate: 6.67%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\237.txt
6839@2022-06-23 19:01:44--6874@2022-06-24 08:26:04
6920@2022-06-24 09:20:30--6953@2022-06-24 10:01:21
loss rate: 1.89%
c:\Users\phosense\Downloads\粤万润房间数据_cv整理\脚本处理数据\data\266.txt
582@2022-06-23 11:39:15-